# RAG With llama-index  + Milvus + LLama

References
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusIndexDemo/
- https://docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/milvus/?h=milvusvectorstore#llama_index.vector_stores.milvus.MilvusVectorStore

## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

MY_CONFIG.DB_URI = './rag_2_llamaindex.db'
MY_CONFIG.COLLECTION_NAME = 'docs_llamaindex'


## Step-2: Setup Embeddings

In [2]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/apps/anaconda3/envs/data-prep-kit-rag-workshop-dev3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Step-3: Connect to Milvus

In [4]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=False
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", MY_CONFIG.DB_URI )

✅ Connected Llama-index to Milvus instance:  ./rag_2_llamaindex.db


## Step-4: Load Document Index from DB

In [5]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context)

print ("✅ Loaded index from vector db:", MY_CONFIG.DB_URI )

✅ Loaded index from vector db: ./rag_2_llamaindex.db
CPU times: user 266 ms, sys: 16.5 ms, total: 283 ms
Wall time: 279 ms


## Step-5: Setup LLM

In [6]:
from llama_index.llms.replicate import Replicate
from llama_index.core import Settings

llm = Replicate(
    model= MY_CONFIG.LLM_MODEL,
    temperature=0.1
)

Settings.llm = llm

## Step-6: Query

In [10]:
query_engine = index.as_query_engine()
res = query_engine.query("What is the target inflation rate?")
print(res)



Based on the provided context information, the target inflation rate is 2 percent.


In [12]:
query_engine = index.as_query_engine()
res = query_engine.query("Which members voted?")
print(res)



Based on the provided context information, the members who voted are:

1. Jerome H. Powell, Chair
2. John C. Williams, Vice Chair
3. Thomas I. Barkin
4. Michael S. Barr
5. Raphael W. Bostic
6. Lisa D. Cook
7. Mary C. Daly
8. Beth M. Hammack
9. Philip N. Jefferson
10. Adriana D. Kugler
11. Christopher J. Waller

Additionally, Austan D. Goolsbee voted as an alternate member at one of the meetings. Michelle W. Bowman voted against the action, preferring to lower the target range for the federal funds rate by 1/4 percentage point.


In [9]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the moon landing?")
print(res)



I'm happy to help! However, I don't see any information about the moon landing in the provided context. The context appears to be about Federal Open Market Committee (FOMC) meetings and monetary policy decisions. Therefore, I cannot answer the query about the moon landing as it is not relevant to the provided context. If you have any other questions or queries related to the FOMC meetings or monetary policy, I'll do my best to assist you!
